# Chromosome 22 - version2 
## It contains the correct 5'3'primer sequences and the right coordinates


## 9.5.2024 (m.d.y)

### Chromosome assemblies

1. [https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/38p14_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/)

In [2]:
# GLOBALS
import sys
import json
import gzip
from hashlib import sha256
import os

inputPath = 'C:\\Users\\creeperpandatrex\\Documents\\1000genomes\\DATA\\GCF000001405-38p14-assembly\\'
chrom22 = "chr22.fna.gz"
inputFile = chrom22

In [6]:
# my is a string of chromosome 'NN' fasta files with 80 char per line to fasta with 2 lines ONLY
CURRENT_CHROMO = 22
def fasta2one (myPath):
    with myPath as file:
        build = ''
        counter = 0
        print(myPath)
        for i in file:
            if counter != 0: # it gets rid of the first line with the header (number of chars is one nucl different)
                build += i.strip() # it builds the pure sequence without extra chars: only A G T C N
            counter = 1
        return build

# my = fasta2one(gzip.open(inputPath + inputFile, 'rt')) # 6.27.2024 __23__ minutes ?
my = fasta2one(gzip.open(inputPath + chrom22, 'rt')) # 9.4.2024 (m.d.y) in 58 seconds or 1 min 48 sec after adding the if statement

print(type(my)) # <class 'str'> fasta2one is a string of ONLY nucleotides, no header


<_io.TextIOWrapper name='C:\\Users\\creeperpandatrex\\Documents\\1000genomes\\DATA\\GCF000001405-38p14-assembly\\chr22.fna.gz' encoding='utf-8'>
<class 'str'>


In [ ]:
buildHeader = '' # header no more since added if statement in cell 6
index = 0
for i in my:
    buildHeader += my[index]
    if index == 68:  # N's start from 69 longest and 68 shortest
        break
    index += 1
print(len(my)) # 50,818,537 50,818,468
print(buildHeader)

In [10]:
# tmp list from set of grep_gene to remove duplicates
myLocal = inputPath + 'awk_ID_symbol_start_end_chr_' + str(CURRENT_CHROMO) # chromosome 22
def idSymbolBuildSet(myPath):
    print(myPath)
    myList = []
    obj = open(myPath, 'r')
    for i in obj:
        local = i.split()
        myList.append(local)
        local = []
    print(len(myList)) # 1523
    mySet = set(tuple(i) for i in myList) # remove duplicates 1523 - 1329 = 
    set2list = list(mySet) # reconvert to a list not a set anymore 
    return set2list

tmp = idSymbolBuildSet(myLocal) # it is now a list (array)
print(len(tmp))

myRange = 10
counter = 0
for i in range(myRange):
    print(tmp[counter])
    counter += 1

# https://stackoverflow.com/questions/26783326/how-to-make-a-set-of-listslist
# >>> l = [[1, 2, 3], [2, 4, 5], [1, 2, 3], [2, 4, 5]]
# >>> set(tuple(i) for i in l)
# {(1, 2, 3), (2, 4, 5)}

C:\Users\creeperpandatrex\Documents\1000genomes\DATA\GCF000001405-38p14-assembly\awk_ID_symbol_start_end_chr_22
1523
1329
('29775', 'CARD10', '37490362', '37519415')
('123706513', 'TRABD-AS1', '50199088', '50200783')
('105373100', 'LOC105373100', '50735829', '50738169')
('29796', 'UQCR10', '29767369', '29770413')
('6663', 'SOX10', '37972312', '37984555')
('100421121', 'LOC100421121', '20639072', '20650483')
('102723780', 'FRG1FP', '10940597', '10961529')
('117134598', 'FAM246A', '21359596', '21361299')
('105373031', 'LOC105373031', '38570500', '38571358')
('150165', 'XKR3', '16783480', '16825411')


In [16]:
def processChrWithIdSymbolSanityCheck():
    myListOfDicMap = [] # array of {'fourTuple': localTuple, 'fourTupleHash':localTupleHash}
    myListIdSymbSeqHash = [] # array of { localTupleHash : seqHash }
    sanityCheck = [] # List of sequence nucleotides at each gene position
    primer = ''
    for i in range(len(tmp)): # list to set to list to remove duplicates, now it is a list-array
        localArray = []
        mySeed = os.getenv('SEED')
        myDicMap = {}
        myDicTupleSeqHash = {} # { localTupleHash:seqHash }
        localTuple = tmp[i]
        localTupleHash = sha256((convertTuple(localTuple) + mySeed).encode()).hexdigest()
        myDicMap.update({'fourTuple':localTuple, 'fourTupleHash':localTupleHash})
        myListOfDicMap.append(myDicMap)
        seqHash = buildSeqHash(localTuple, mySeed) # [myseqhash, myseq], 
        myDicTupleSeqHash.update( {localTupleHash: seqHash[0] })
        # print(type(seqHash[1]))
        seqSize = len(seqHash[1])
        # print('>'+str(localTuple)+'\t'+str(seqSize)+'\n')
        primer += '>'+str(localTuple)+'\t'+str(seqSize)+'\n'
        first = ''
        last = ''
        counter = 0
        if len(seqHash[1]) > 40:
            for j in range(40):
                first += seqHash[1][counter]
                counter += 1
            localArray.append(first)
        counter =  -40
        if len(seqHash[1]) > 40:
            for k in range(40):
                last += (seqHash[1][counter])
                counter += 1
            localArray.append(last)
        # print(first+'..'+len(seqHash)+'..'+last)   #### PRINT
        # print(first+'....'+last)   #########  PRINT
        primer += first+'....'+last+'\n'
        sanityCheck.append(seqHash[1])
        myListIdSymbSeqHash.append(myDicTupleSeqHash)
    return [myListOfDicMap, myListIdSymbSeqHash, sanityCheck, primer]    
    
def print2file (myPath, result):
    out = open(myPath, 'w')
    out.write(result) # it must be a string

def buildSeqHash(tuple, mySeed):
    mySeq = ''
    id,symb,start,end = tuple
    start = int(start) - 1
    end = int(end) 
    # counter = start + 80
    # counter = start + inputIndex[CURRENT_CHROMO]
    counter = start
    for i in range(end - start):
        mySeq += my[counter] # it is a string of nucleotides from chromosome22 MY MY MY MY MY
        counter += 1
    mySeqHash = sha256(((mySeq+mySeed)).encode()).hexdigest()
    return [mySeqHash, mySeq] # both string


# print(buildSeq(processing[0]['fourTuple']))
        
def convertTuple(tup):  # the hashing command accepts only strings
    str = ''.join(tup)
    return str

processing = processChrWithIdSymbolSanityCheck()
print(len(processing)) # 4; 2 minutes



counter = 0
# for i in range(1):
    # print(processing[0][counter]) # ext: labelmap array of { 'fourTuple': (id,symb,start,end), 'fourTupleHash': hash } 
    # print(processing[1][counter]) # ext: labelseqhash array of { fourTuplehash: seqHash }
    # print(processing[2][counter]) # gene sequence in array
    # print(len(processing[3])) # primer string not a real array

    # counter += 1

# {'fourTuple': ('105373037', 'LOC105373037', '40555177', '40564466'), 'fourTupleHash': 'b1514454efb34d796e695624a103c9a5b4727f09df995728389633123747a653'}
# {'fourTuple': ('339685', 'LOC339685', '47345571', '47373541'), 'fourTupleHash': 'b5c159d961591a4e83f3726b4933d92890392c769b28c825c4bc0b2d05bca5d8'}


4
183959


In [22]:
def printCharOfString(theStr, num):
    build = ''
    for i in range(num):
        build += theStr[i]
    print(build)

printCharOfString(processing[3], 210)

>('29775', 'CARD10', '37490362', '37519415')	29054
TCTATGACAAAAGTGTTTAATGCTCCATAATGTGCAGCTT....CCGCACTCGGGCGGCGGCTCCGCCGGCGCAGGGGGGCGGT
>('123706513', 'TRABD-AS1', '50199088', '50200783')	1696
TCTTTTACTCAGGTAAT


In [23]:
# 
# print2file(inputPath + inputFile + ".labelmap", json.dumps(dict2hash(globalDictionary)[0]))
# print2file(inputPath + inputFile + ".labelseqhash", json.dumps(dict2hash(globalDictionary)[1]))
result = processing

def print2file (myPath, result):
    out = open(myPath, 'w')
    out.write(result) # it must be a string

def print2gz (myPath, result, addExt):
   with gzip.open(myPath + addExt, 'w') as f:
    f.write((json.dumps(result)).encode()) # no decode() since it is a str already

print(inputPath+inputFile)
print2gz(inputPath + inputFile + 'v2', result[0], '.labelmap.gz')
print2gz(inputPath + inputFile + 'v2', result[1], '.labelseqhash.gz')

C:\Users\creeperpandatrex\Documents\1000genomes\DATA\GCF000001405-38p14-assembly\chr22.fna.gz


In [21]:
### this is SANITY CHECK  
### processing[0] is .labelmap list of dics json
### processing[1] is  .labelseqhash list of dics json
### processing[2] is .fna.gene array of sequences from gene
### processing[3] is .primer string 2 lines 

print(len(processing[2])) # 1329

print(inputPath+inputFile)

print2file(inputPath+inputFile+'.primer', processing[3])
# print2file(inputPath+inputFile+'.fna.gene', json.dumps(processing[2]))  # TOO BIG

1329
C:\Users\creeperpandatrex\Documents\1000genomes\DATA\GCF000001405-38p14-assembly\chr22.fna.gz


In [24]:
print(len(processing[2]))

1329
